# Intro to gragh convolutional surrogate models
Eamon Whalen

In [1]:
import sys
import numpy as np
import pandas as pd
import altair as alt

sys.path.append('./models')
from feastnetSurrogateModel import FeaStNet

sys.path.append('./readers')
from loadGhGraphs import loadGhGraphs

sys.path.append('./visualization')
from altTrussViz import plotTruss, interactiveErrorPlot

sys.path.append('./util')
from gcnSurrogateUtil import *

## 1. Load simulation data

In [2]:
doeFile = "/home/ewhalen/projects/data/trusses/2D_Truss_v1.3/design_7_N_1000.csv"
allGraphsUnfiltered = loadGhGraphs(doeFile, NUM_DV=5)

maxes = [max(np.abs(graph.y.numpy().flatten())) for graph in allGraphsUnfiltered]
source = pd.DataFrame(maxes, columns=['maxes'])
source.describe()

,maxes
count,1000.000000
mean,0.199516
std,2.524026
min,0.006570
25%,0.015425
50%,0.024334
75%,0.046981
max,67.325867


In [3]:
plotTruss(allGraphsUnfiltered[0], showDeformed=True, defScale=100)

alt.LayerChart(...)

## 2. Filter and partition

In [4]:
allGraphs = filterbyDisp(allGraphsUnfiltered, 0.9)
trainData, valData, testData = partitionGraphList(allGraphs)

maxes = [max(np.abs(graph.y.numpy().flatten())) for graph in allGraphs]
source = pd.DataFrame(maxes, columns=['maxes'])
source.describe()

,maxes
count,900.000000
mean,0.028952
std,0.019071
min,0.006570
25%,0.014951
50%,0.021719
75%,0.038367
max,0.097861


## 3. Train a GCN

In [5]:
gcn = FeaStNet()
history = gcn.trainModel(trainData, valData, 
                     epochs=100, 
                     batchSize=256, 
                     flatten=True, 
                     logTrans=False, 
                     ssTrans=True, 
                     saveDir='./results/gcn01/')

plotHistory(history)

epoch: 0   trainLoss: 9.3584e-01   valLoss:9.4250e-01  time: 1.35e+00
epoch: 1   trainLoss: 9.0147e-01   valLoss:9.3961e-01  time: 1.05e+00
epoch: 2   trainLoss: 8.6180e-01   valLoss:9.3875e-01  time: 1.04e+00
epoch: 3   trainLoss: 7.7282e-01   valLoss:9.3933e-01  time: 1.03e+00
epoch: 4   trainLoss: 7.6027e-01   valLoss:9.5182e-01  time: 1.06e+00
epoch: 5   trainLoss: 7.2778e-01   valLoss:9.3323e-01  time: 1.04e+00
epoch: 6   trainLoss: 6.8650e-01   valLoss:9.0575e-01  time: 1.01e+00
epoch: 7   trainLoss: 6.7977e-01   valLoss:8.4310e-01  time: 1.01e+00
epoch: 8   trainLoss: 6.3048e-01   valLoss:7.4903e-01  time: 1.01e+00
epoch: 9   trainLoss: 5.9061e-01   valLoss:6.6651e-01  time: 1.01e+00
epoch: 10   trainLoss: 6.1091e-01   valLoss:6.1901e-01  time: 1.00e+00
epoch: 11   trainLoss: 5.5591e-01   valLoss:5.8819e-01  time: 9.95e-01
epoch: 12   trainLoss: 4.9625e-01   valLoss:5.6279e-01  time: 9.95e-01
epoch: 13   trainLoss: 4.8525e-01   valLoss:5.4435e-01  time: 1.00e+00
epoch: 14   trai

alt.Chart(...)

## 4. Test the GCN

In [6]:
trainRes = gcn.testModel(trainData)
testRes = gcn.testModel(testData)
pd.DataFrame([trainRes, testRes], index=['train', 'test'])

,mse,mae,mre,peakR2,maxAggR2,meanAggR2,minAggR2
train,0.000011,0.002209,0.102437,0.939338,0.971587,0.452770,-5.934452
test,0.000018,0.002616,0.119010,0.833666,0.927007,0.354463,-6.426411


## 5. Visualize some predictions

In [7]:
i = 9
pred = gcn.predict([testData[i]])[0]
plotTruss(testData[i], showDeformed=True, defScale=100, prediction=pred)

alt.LayerChart(...)

In [8]:
# interactive scatter plot
alt.data_transformers.enable('json')
allPreds = gcn.predict(testData)
display(interactiveErrorPlot(testData, allPreds))
alt.data_transformers.enable('default');

alt.HConcatChart(...)